In [1]:
print("HEllo")

HEllo


In [8]:
import os

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

from dotenv import load_dotenv, find_dotenv

In [7]:
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']
tavily_api_key = os.environ['TAVILY_API_KEY']

In [9]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile"
)

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
res = search.invoke("Who is the persident of Israel in 2025")

In [11]:
print(res)

[{'url': 'https://www.reuters.com/video/watch/idRW900105022025RP1/', 'content': "Article-Playlist: Middle East. Israel's president praises Trump-Netanyahu meeting, avoids Gaza. February 5, 2025. Israeli President Isaac Herzog on"}, {'url': 'https://x.com/israelipm?lang=en', 'content': 'Ronen Bar will conclude his duties on 10 April 2025 or when a permanent ISA Director is appointed – whichever comes first.'}]


In [13]:
tools = [search]

In [14]:
tools

[TavilySearchResults(max_results=2)]

In [15]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model=llm , tools=tools)

In [16]:
from langchain_core.messages import HumanMessage
human_massage = HumanMessage(content="Where is the soccer Eurocup 2024 played?")
response = agent_executor.invoke({"messages" : [human_massage]})

In [ ]:
response["messages"] ## return response from live URL

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='7dc1d0ef-931a-459a-92cb-fc79157e6f64'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f7m3', 'function': {'arguments': '{"query": "Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 289, 'total_tokens': 312, 'completion_time': 0.083636364, 'prompt_time': 0.024905264, 'queue_time': 0.101293075, 'total_time': 0.108541628}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_8dd9fca28c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2c8bb6a7-7521-4bfc-8e83-cb597f9c7e25-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Eurocup 2024 location'}, 'id': 'call_f7m3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 289, 'output_tokens': 23, 'total_tokens': 312}),
 ToolMessage(content='[{"url": "https://www.forbes.com/sites/bradadgate/

In [20]:
human_massage = HumanMessage(content="What is Python?")
response = agent_executor.invoke({"messages" : [human_massage]})
response["messages"]

[HumanMessage(content='What is Python?', id='3dbd38ee-1d0e-4e84-a7a8-5876942c82c8'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3m9x', 'function': {'arguments': '{"query": "What is Python?"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 563, 'total_tokens': 583, 'completion_time': 0.072727273, 'prompt_time': 0.043265058, 'queue_time': -0.109759086, 'total_time': 0.115992331}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_8dd9fca28c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-09b46988-4264-4b93-b3b7-7acc3685675d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'What is Python?'}, 'id': 'call_3m9x', 'type': 'tool_call'}], usage_metadata={'input_tokens': 563, 'output_tokens': 20, 'total_tokens': 583}),
 ToolMessage(content='[{"url": "https://www.w3schools.com/python/python_intro.asp", "content": "What is Pyth

In [21]:
human_massage = HumanMessage(content="Where is the soccer Eurocup 2024 played?")
for chunk in agent_executor.stream(
    {"messages" : [human_massage]}
    
): 
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ezdn', 'function': {'arguments': '{"query": "Eurocup 2024 host country"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 289, 'total_tokens': 313, 'completion_time': 0.087272727, 'prompt_time': 0.018331319, 'queue_time': 0.100541796, 'total_time': 0.105604046}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_90c1d253ff', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-51c5128f-b1d1-409d-962a-2944077dfe36-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Eurocup 2024 host country'}, 'id': 'call_ezdn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 289, 'output_tokens': 24, 'total_tokens': 313})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.forbes.com/sites/bradadgate/2024/07/03/where-to-watch-euro-cup-2024/", "conte

### Adding Memory like Conversation

In [23]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent_executor = create_react_agent(model=llm , tools=tools, checkpointer=memory)
config = {"configurable" : {"thread_id" : "001}"}} ## specific user --> if you change he open a new window 

In [26]:
human_massage = HumanMessage(content="Where is the soccer Eurocup 2024 played?")
for chunk in agent_executor.stream(
    {"messages" : [human_massage]}, config=config
    
): 
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_t8dt', 'function': {'arguments': '{"query": "soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 289, 'total_tokens': 314, 'completion_time': 0.090909091, 'prompt_time': 0.018370068, 'queue_time': 0.1017295, 'total_time': 0.109279159}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_41c250edc7', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8bae9af4-1983-469a-9f6a-5a927a0b2939-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'soccer Eurocup 2024 location'}, 'id': 'call_t8dt', 'type': 'tool_call'}], usage_metadata={'input_tokens': 289, 'output_tokens': 25, 'total_tokens': 314})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024_final", "content": "The final was held a

In [27]:
human_massage = HumanMessage(content="who won the 2024 soccer Eurocup?")
for chunk in agent_executor.stream(
    {"messages" : [human_massage]}, config=config
    
): 
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zx3x', 'function': {'arguments': '{"query": "2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 944, 'total_tokens': 967, 'completion_time': 0.083636364, 'prompt_time': 0.060558123, 'queue_time': 0.106881315, 'total_time': 0.144194487}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c4760ee73b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-624f42dc-575e-438b-9e34-ce4d79b7f80c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_zx3x', 'type': 'tool_call'}], usage_metadata={'input_tokens': 944, 'output_tokens': 23, 'total_tokens': 967})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.cnn.com/2024/07/14/sport/spain-england-euro-2024-final-spt-intl/index.html"